# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
import pandas as pd
import numpy as np

import re
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

from lightgbm import LGBMClassifier

## 1 Подготовка

Загрузим данные и посмотрим на них.

In [2]:
df = pd.read_csv('datasets/toxic_comments.csv')
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Таблица содержит тексты на английском языке и информацию о том, являются ли эти тексты токсичными.

In [3]:
print('Варианты значений столбца toxic:', df['toxic'].unique())
print('Количество положительных объектов: {:.0%}'.format(df['toxic'].mean()))
print()
df.info()

Варианты значений столбца toxic: [0 1]
Количество положительных объектов: 10%

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Пропущенных значений, ошибок в целевом признаке или проблем с типами в талице нет. Данные несбалансированные: положительных объектов только 10%.

Очистим текст от лишних символов: оставим только символы английского алфавита. После этого проведём лемматизацию английского текста. Стоп-слова оставим в их исходном виде, чтобы учесть при создании векторов.

In [4]:
stemmer = SnowballStemmer('english', ignore_stopwords=True)

def only_text(text, stemmer=stemmer):
    '''
    Оставляет в тексте только слова, написанные латиницей.
    Лемматизирует английский текст.
    '''
    
    clean_text = re.sub(r'[^a-zA-Z]', ' ', text)
    clean_text = ' '.join([stemmer.stem(word) for word in clean_text.split()])
    return clean_text

In [5]:
%%time

df['lemm_text'] = list(map(lambda x: only_text(x), df['text'].values))

CPU times: user 51.9 s, sys: 50.1 ms, total: 51.9 s
Wall time: 51.9 s


Посмотрим на несколько примеров текстов до и после преобразования.

In [6]:
print(df['text'].sample(n=3, random_state=42).values)

["Geez, are you forgetful!  We've already discussed why Marx  was  not an anarchist, i.e. he wanted to use a State to mold his 'socialist man.'  Ergo, he is a statist - the opposite of an  anarchist.  I know a guy who says that, when he gets old and his teeth fall out, he'll quit eating meat.  Would you call him a vegetarian?"
 'Carioca RFA \n\nThanks for your support on my request for adminship.\n\nThe final outcome was (31/4/1), so I am now an administrator. If you have any comments or concerns on my actions as an administrator, please let me know. Thank you!'
 '"\n\n Birthday \n\nNo worries, It\'s what I do ;)Enjoy ur day|talk|e "']


In [7]:
print(df['lemm_text'].sample(n=3, random_state=42).values)

['geez are you forget we ve alreadi discuss why marx was not an anarchist i e he want to use a state to mold his socialist man ergo he is a statist the opposit of an anarchist i know a guy who say that when he get old and his teeth fall out he ll quit eat meat would you call him a vegetarian'
 'carioca rfa thank for your support on my request for adminship the final outcom was so i am now an administr if you have any comment or concern on my action as an administr pleas let me know thank you'
 'birthday no worri it s what i do enjoy ur day talk e']


При кросс-валидации обучающая выборка каждый раз меняется, поэтому для каждой итерации нужно заново использовать трансформер, создающий векторы из слов. Добавим TfidfVectorizer в пайплайн. Для создания векторов будем использовать важность слов и биграмм.

In [8]:
stop_words = set(stopwords.words('english'))

def get_pipline(model, stop_words=stop_words):
    '''
    Создаёт пайплайн: считает оценки важности слов в лемматизированных текстах, 
    передаёт получившиеся векторы модели.
    '''
    pipeline = Pipeline(steps=[('count_tf_idf', TfidfVectorizer(stop_words=stop_words, ngram_range=(1,2))),
                               ('model', model)])
        
    return pipeline

Создадим функцию, которая будет находить лучшие параметры для модели и считать F1.

In [9]:
def search_params(model, params, X, y):
    '''Ищет лучшие параметры модели с помощью GridSearch. Возвращает лучшие параметры и F1.'''
    
    kf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    grid_search = GridSearchCV(
        model, params, 
        scoring = 'f1', 
        cv = kf,
        n_jobs = -1
    )
    grid_search.fit(X, y)
    
    print('Лучшие параметры:', grid_search.best_params_)
    print('F1 = {:.3f}'.format(grid_search.best_score_))
    print()
    return grid_search.best_estimator_, grid_search.best_score_

**Вывод**

Данные загружены и изучены. Тексты написаны на английском языке, присутствует дисбаланс классов.

Все тексты очищены от лишних символов и лемматизированы. Написана функция для создания паплайна: 1) преобразование текстов в векторы с использованием важности слов и биграмм, 2) модель машинного обучения. Стоп-слова не учитываются. Также написана функция для поиска лучших гиперпараметров на кросс-валидации и расчёта метрики F1.

## 2 Обучение

Для ориентира посмотрим, какое значение метрики можно получить на dummy-модели. 

In [10]:
%%time

model_dm = DummyClassifier(random_state=42)
params_dm = {'model__strategy': ['stratified', 'most_frequent']}
pipeline_dm = get_pipline(model_dm)
pipeline_dm, f1_dm = search_params(pipeline_dm, params_dm, df['lemm_text'], df['toxic'])

Лучшие параметры: {'model__strategy': 'stratified'}
F1 = 0.097

CPU times: user 14.8 s, sys: 816 ms, total: 15.7 s
Wall time: 43.8 s


Обучим логистическую регрессию. Т.к. классы несбалансированы, попробуем вариант class_weight='balanced'.

In [11]:
%%time

model_lr = LogisticRegression(random_state=42, solver='liblinear')

params_lr = {
    'model__penalty': ['l1', 'l2'],
    'model__C': [1, 3],
    'model__class_weight': [None, 'balanced']
}

pipeline_lr = get_pipline(model_lr)
pipeline_lr, f1_lr = search_params(pipeline_lr, params_lr, df['lemm_text'], df['toxic'])

Лучшие параметры: {'model__C': 3, 'model__class_weight': None, 'model__penalty': 'l1'}
F1 = 0.798

CPU times: user 24.6 s, sys: 5.06 s, total: 29.6 s
Wall time: 22min 52s


Лучшее значение F1 получено без учёта дисбаланса, т.е. при class_weight=None. 

Рассмотрим градиентный бустинг.

In [12]:
%%time

model_gb = LGBMClassifier(random_state=42)

params_gb = {
    'model__n_estimators': [1000, 1500],
    'model__max_depth': [10, 5]
}

pipeline_gb = get_pipline(model_gb)
pipeline_gb, f1_gb = search_params(pipeline_gb, params_gb, df['lemm_text'], df['toxic'])

Лучшие параметры: {'model__max_depth': 10, 'model__n_estimators': 1500}
F1 = 0.789

CPU times: user 50min 2s, sys: 38 s, total: 50min 40s
Wall time: 29min 42s


У бустинга значение F1 получилось меньше, но оно тоже превышает заданную планку в 0.75.

**Вывод**

Для ориентира и оценки адекватности посчитано значение F1 для dummy-модели. Рассмотрены логистическая регрессия и градиентный бустинг с разным количеством гиперпараметров. Лучшие варианты обеих моделей показали значение F1 больше 0.75 и существенно превзошли dummy-модель. Наибольшее значение метрики у логистической регрессии.

## 3 Выводы

В работе использован набор объектов: текст на английском языке и метка класса: 1 или 0 - токсичный текст или нет. Положительных объектов 10%. Все тексты очищены от лишних символов, лемматизированы и преобразованы в векторы по значению важности слов и биграмм без учёта стоп-слов. 

Рассмотрены модели логистической регрессии и градиентного бустинга. На кросс-валидации выбраны лучшие гиперпараметры и посчитано значение F1. Кроме того, значение метрики посчитано для dummy-модели. Логистическая регрессия показала лучшие результаты. 

In [13]:
pd.DataFrame({
    'model': ['DummyClassifier', 'LogisticRegression', 'LGBMClassifier'],
    'F1': ['{:.3f}'.format(f1_dm), '{:.3f}'.format(f1_lr), '{:.3f}'.format(f1_gb)]
})

,model,F1
0,DummyClassifier,0.097
1,LogisticRegression,0.798
2,LGBMClassifier,0.789


## 4 Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны